In [46]:
import boto3, json
from pprint import pprint

## Credentials

In [47]:
def getAWSCreds():
    credentials = json.load(open('../esrcd.json'))
    awsCreds = credentials['aws']
    return(awsCreds)

## Sessions and Resources

In [89]:
def createSession():
    session = boto3.session.Session(aws_access_key_id=getAWSCreds()['accessKey'], 
                                    aws_secret_access_key=getAWSCreds()['secretKey'])
    return(session)

def createResourceObj(resourceName, regionName):
    resource = boto3.resource(resourceName, region_name=regionName, 
                              aws_access_key_id=getAWSCreds()['accessKey'], 
                              aws_secret_access_key=getAWSCreds()['secretKey'])
    return(resource)

def createClient(resourceName,regionName):
    client = boto3.client(resourceName, region_name=regionName,
                          aws_access_key_id=getAWSCreds()['accessKey'], 
                          aws_secret_access_key=getAWSCreds()['secretKey'])
    return(client)

## Regions and Services

In [49]:
def getAllRegions(session):
    return(session.get_available_regions('ec2', partition_name='aws'))

def getAllServices(session):
    return(session.get_available_services())

In [50]:
def getAllServicesAndRegions(session):
    allServices = getAllServices(session)
    serviceDict = {}
    for service in allServices:
        availableRegions = session.get_available_regions(service, partition_name='aws')
        serviceDict[service] = availableRegions
    return(serviceDict)

In [139]:
def getIncludedRegions(excludedRegions, session):
    allRegions = getAllRegions(session)
    return([region for region in allRegions if region not in excludedRegions])

def getIncludedServices(excludedServices, session):
    allServices = getAllServices(session)
    return([service for service in allServices if service not in excludedServices])

def getShortlistedRegions(includedRegions, validRegions):
    return([region for region in includedRegions if region in validRegions])

def getAllRegionsForService(serviceName, session):
    return(session.get_available_regions(serviceName, partition_name='aws'))

## EC2 Instances

In [119]:
def deleteEC2Instances(includedRegions):
    instanceIds = listEC2InstanceIds(includedRegions)
    for region in instanceIds.keys():
        ec2 = createResourceObj('ec2', region)
        for instanceId in instanceIds[region]:
            instance = ec2.Instance(id=instanceId).terminate(DryRun=False)
            print("Deleted!")

def listEC2InstanceIds(includedRegions):
    instanceIds = {}
    for region in includedRegions:
        ec2 = createResourceObj('ec2', region)
        instances = ec2.instances.filter(
        Filters=[{'Name': 'instance-state-name', 'Values': ['running','stopped']}])
        for instance in instances:
            if(region not in instanceIds.keys()):
                instanceIds.setdefault(region,[]).append(instance.id)
            else:
                instanceIds[region].append(instance.id)
    pprint(instanceIds)
    return(instanceIds)

## SNS

In [120]:
def deleteTopics(includedRegions, session):
    snsRegions = getShortlistedRegions(includedRegions, getAllRegionsForService('sns', session))
    for region in snsRegions:
        print("Region: {}".format(region))
        sns = createResourceObj('sns', region)
        for topic in sns.topics.all():
            print("Found topics!")
            response = topic.delete()
            if(response['ResponseMetadata']['HTTPStatusCode'] != 200):
                print("Something went wrong")
                pprint(response)
            else:
                print("Deleted:")
                print("++++++++++")
                pprint(response)
                print("++++++++++")
    print("All done!")

## S3 Buckets

In [121]:
def getExcludedBucketNames():
    return('chiragbilling chiragwebsite'.split())

def deleteS3Buckets():
    s3 = createResourceObj('s3', 'ap-south-1')
    validBuckets = []
    for bucket in s3.buckets.all():
        print(bucket.name)
        if(bucket.name not in getExcludedBucketNames()):
            print("Added bucket")
            validBuckets.append(bucket)
    Delete = {'Objects':[]}
    for bucket in validBuckets:
        for page in bucket.objects.pages():
            for obj in page:
                print(obj.key)
                Delete['Objects'].append({
                    'Key' : obj.key
                })
            bucket.delete_objects(Delete=Delete)
            print("Page complete. Deleted objects")
        print("All pages complete. Deleting bucket now")
        bucket.delete()
        print("Bucket deleted!")

## DynamoDB

In [152]:
def deleteDynamoTables(includedRegions, session):
    validRegions = getShortlistedRegions(includedRegions, getAllRegionsForService('dynamodb', session))
    for region in validRegions:
        print("Region: {}".format(region))
        dynamo = createResourceObj('dynamodb', region)
        tableList = []
        for table in dynamo.tables.all():
            tableList.append(table)
        for table in tableList:
            response = table.delete()
            if(response['ResponseMetadata']['HTTPStatusCode']!=200):
                print("Something went wrong!")
                #Add twilio code here
            else:
                print("Success!")
                pprint(response)
    print("All done!")

## SQS

In [170]:
def deleteSQS(includedRegions):
    validRegions = getShortlistedRegions(includedRegions, getAllRegionsForService('sqs', session))
    qlist = []
    for region in validRegions:
        sqs = createResourceObj('sqs', region)
        for q in sqs.queues.all():
            response = q.delete()
            if(response['ResponseMetadata']['HTTPStatusCode']!=200):
                print("Something went wrong!")
                pprint(response)
                #Add twilio code here
            else:
                print("Success!")
                pprint(response)
    print("SQS: All done!")

# Not Tested:

In [23]:
def deleteVolumes(includedRegions, ec2Client):
    response = ec2Client.describe_volumes(Filters=[{'Name' : 'attachment.status', 'Values' : ['detached']}])
    volumeIds = getVolumeIds(response)
    for volumeId in volumeIds:
        ec2Client.delete_volume(VolumeId=volumeId,DryRun=False)

************************************************************************************************************

## Testing Section

In [180]:
excludedRegions = ['us-east-1']
excludedServices = ['cloudwatch', 'budgets']

In [182]:
session = createSession()
# allRegions = getAllRegions(session)
# allServicesRegions = getAllServicesAndRegions(session)
# allServices = getAllServices(session)
includedRegions = getIncludedRegions(excludedRegions, session)
includedServices = getIncludedServices(excludedServices, session)
# ec2Regions = getAllRegionsForService('ec2', session)
# ec2 = createResourceObj('ec2', 'ap-south-1')
# ec22 = createResourceObj('ec2', 'sa-east-1')

In [72]:
resourceObjCompatibleServices = "cloudformation cloudwatch dynamodb ec2 glacier iam opsworks s3 sns sqs".split()
resourceObjCompatibleServices

['cloudformation',
 'cloudwatch',
 'dynamodb',
 'ec2',
 'glacier',
 'iam',
 'opsworks',
 's3',
 'sns',
 'sqs']

In [186]:
opsWorks = createResourceObj('opsworks', 'ap-south-1')

In [187]:
stacks = []
for stack in opsWorks.stacks.all():
    stacks.append(stack)

In [188]:
stack = stacks[0]

In [189]:
deleteEC2Instances(includedRegions)

{}


In [194]:
stack.Summary()

opsworks.StackSummary(stack_id='d3755604-b472-465a-8420-42178319b933')

In [196]:
for layer in stack.layers.all():
    print(layer)

opsworks.Layer(id='a4534a83-5672-47bd-83ec-bc62c224a4ff')
